# rCSI time-series dataset released by WFP

The complete version of the data released by WFP regarding the Reduced Coping Strategy Index (rCSI).

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [1]:
from ipywidgets import interact, widgets, fixed
import plotly.graph_objects as go
import seaborn as sns
from scipy.interpolate import splrep, splev
import pandas as pd
import numpy as np
import ntpath
import glob

In [2]:
# Read the data released by wfp regarding the rCSI indicator.
path = "./wfp_data/"
all_files = glob.glob(path + "*.csv")

dfs = []

for filename in all_files:
    df = pd.read_csv(filename)
    df["Country"] = ntpath.basename(filename).split(".")[0]
    dfs.append(df)

In [3]:
print("The data released by wfp:")
df = pd.concat(dfs, axis = 0, ignore_index = True)
df.head()

The data released by wfp:


,SvyDate,Date,Dmgrph,DmgrphCode,Mean_crrnt,Pop,PopNmbr,Country
0,2019-7-15_2019-9-8,2019-09-08,BOUCLE-DU-MOUHOUN,900712.0,19.499504,1976217.0,385352.0,Burkina Faso
1,2019-7-15_2019-9-8,2019-09-08,CASCADES,900713.0,8.430561,822445.0,69336.0,Burkina Faso
2,2019-7-15_2019-9-8,2019-09-08,CENTRE,900714.0,7.589967,2854356.0,216644.0,Burkina Faso
3,2019-7-15_2019-9-8,2019-09-08,CENTRE-EST,900715.0,20.051049,2854356.0,572328.0,Burkina Faso
4,2019-7-15_2019-9-8,2019-09-08,CENTRE-NORD,900716.0,18.757613,1687858.0,316601.0,Burkina Faso


### Brief items description

- *SvyDate*: reference period of the data collection.
- *Date*: reference date (i.e. the end of the reference period). 
- *Dmgrph*: administrative area name.
- *DmgrphCode*: a code for identyfing the adminstrata.
- *Mean_crrnt*: corresponds to the % of people with rCSI>=19. 
- *Pop*: area population size.
- *PopNmbr*: number of people with rCSI>=19 (i.e. Mean_crrnt * Pop).

In [4]:
# Check if the dataframe contains NaN values.
print("Check if the dataframe contains NaN values:")
df.isnull().sum()

Check if the dataframe contains NaN values:


SvyDate         0
Date            0
Dmgrph          0
DmgrphCode    524
Mean_crrnt      2
Pop             2
PopNmbr         2
Country         0
dtype: int64

In [5]:
# Let's delete the item 'DmgrphCode'.
df.drop(["DmgrphCode"], axis = 1, inplace = True) 

In [6]:
# Rename some columns.
df.rename(columns = {"Date": "Datetime", "Dmgrph": "AdminStrata", "Mean_crrnt": "Metric"}, inplace = True)

In [7]:
df.head()

,SvyDate,Datetime,AdminStrata,Metric,Pop,PopNmbr,Country
0,2019-7-15_2019-9-8,2019-09-08,BOUCLE-DU-MOUHOUN,19.499504,1976217.0,385352.0,Burkina Faso
1,2019-7-15_2019-9-8,2019-09-08,CASCADES,8.430561,822445.0,69336.0,Burkina Faso
2,2019-7-15_2019-9-8,2019-09-08,CENTRE,7.589967,2854356.0,216644.0,Burkina Faso
3,2019-7-15_2019-9-8,2019-09-08,CENTRE-EST,20.051049,2854356.0,572328.0,Burkina Faso
4,2019-7-15_2019-9-8,2019-09-08,CENTRE-NORD,18.757613,1687858.0,316601.0,Burkina Faso


### Country item

In [8]:
print("The countries are:", ", ".join(df.Country.unique()))

The countries are: Burkina Faso, Nigeria, Syria, Yemen


### AdminStrata item

In [9]:
AdminStratas = df.groupby("Country")["AdminStrata"].unique()

- **Yemen**: the administrative division of Yemen is divided into two main divisions (governorates and districts). There are 22 governorates, including the capital Sana'a (Amanat Al Asimah) and Socotra Archipelago. The rCSI dataframe has the values of all the 22 governorates.

In [10]:
print(AdminStratas["Yemen"].shape)
AdminStratas["Yemen"]

(22,)


array(['Abyan', 'Aden', 'Al Bayda', "Al Dhale'e", 'Al Hudaydah',
       'Al Jawf', 'Al Maharah', 'Al Mahwit', 'Amanat Al Asimah', 'Amran',
       'Dhamar', 'Hadramaut', 'Hajjah', 'Ibb', 'Lahj', 'Marib', 'Raymah',
       "Sa'ada", "Sana'a", 'Shabwah', 'Socotra', 'Taizz'], dtype=object)

- **Nigeria**: Nigeria is divided into 36 states. It also includes *The Federal Capital Territory* that is not a state and it is under the direct control of the federal government. The AdminStrata items of the rCSI dataframe cover 3 states (Adamawa, Borno and Yobe) and each of these state is diveded into three parts: North, South, Central.

In [11]:
print(AdminStratas["Nigeria"].shape)
AdminStratas["Nigeria"]

(9,)


array(['Adamawa Central', 'Adamawa North', 'Adamawa South',
       'Borno Central', 'Borno North', 'Borno South', 'Yobe East',
       'Yobe North', 'Yobe South'], dtype=object)

- **Syria**: the governatores of the Syria country are 14. The rCSI dataframe has the values of 13 governorates (Idlib governatore not included).

In [12]:
print(AdminStratas["Syria"].shape)
AdminStratas["Syria"]

(13,)


array(['Al-Hasakeh', 'Aleppo', 'As-Sweida', 'Damascus', "Dar'a", 'Hama',
       'Homs', 'Lattakia', 'Rural Damascus', 'Tartous', 'Ar-Raqqa',
       'Deir-ez-Zor', 'Quneitra'], dtype=object)

- **Burkina Faso**: Burkina Faso is divided into 13 administrative regions. The rCSI dataframe has the values of all 13 regions.

In [13]:
print(AdminStratas["Burkina Faso"].shape)
AdminStratas["Burkina Faso"]

(13,)


array(['BOUCLE-DU-MOUHOUN', 'CASCADES', 'CENTRE', 'CENTRE-EST',
       'CENTRE-NORD', 'CENTRE-OUEST', 'CENTRE-SUD', 'EST',
       'HAUTS-BASSINS', 'NORD', 'PLATEAU-CENTRAL', 'SAHEL', 'SUD-OUEST'],
      dtype=object)

In [14]:
# The strings of AdminStrata key for Burkina Faso country are uppercase, I change them to obtain lowercase with first capital letter.
def to_lower(country, admin):
    if country == "Burkina Faso":
        admin = admin.lower().title()      
    return admin
    
df["AdminStrata"] = df[["Country", "AdminStrata"]].apply(lambda x: to_lower(*x), axis = 1)

In [15]:
# Check the min e max values of the Metric.
print("The min and max values of the Metric:")
print(df.Metric.min(), ",", df.Metric.max())

The min and max values of the Metric:
0.0 , 99.97458240652935


In [16]:
# Creation of an accurate datetime format (not consider the item 'SvyDate').
df.drop(["SvyDate"], axis = 1, inplace = True) 
df["Datetime"] = pd.to_datetime(df["Datetime"])
df.sort_values("Datetime", ascending = True, inplace = True) # Sort on datetime.
df = df.groupby(["Country", "AdminStrata"]).apply(lambda group: group.set_index("Datetime").resample("D").mean()).reset_index()
df.reset_index(drop = True, inplace = True)
df.head()

,Country,AdminStrata,Datetime,Metric,Pop,PopNmbr
0,Burkina Faso,Boucle-Du-Mouhoun,2019-09-08,19.499504,1976217.0,385352.0
1,Burkina Faso,Boucle-Du-Mouhoun,2019-09-09,19.567379,1976217.0,386693.0
2,Burkina Faso,Boucle-Du-Mouhoun,2019-09-10,20.400684,1976217.0,403161.0
3,Burkina Faso,Boucle-Du-Mouhoun,2019-09-11,20.405577,1976217.0,403258.0
4,Burkina Faso,Boucle-Du-Mouhoun,2019-09-12,19.238527,1976217.0,380195.0


In [17]:
# Create a dataframe with multi index column in order to have a summary dataframe of the time-series.
df.drop(labels = ["Pop", "PopNmbr"], axis = 1, inplace = True)
df = df.set_index(["Datetime", "Country", "AdminStrata"]).unstack(["Country", "AdminStrata"])
df.columns = df.columns.droplevel(0)
df.columns = pd.MultiIndex.from_tuples(list(map(lambda x: tuple(list(x) + ["rCSI"]), df.columns)))
df.columns.rename("Country", level = 0, inplace = True)
df.columns.rename("AdminStrata", level = 1, inplace = True)
df.columns.rename("Indicator", level = 2, inplace = True)
freq = "D"
df.index.freq = freq
df.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                rCSI     rCSI   rCSI       rCSI        rCSI   
Datetime                                                               
2018-08-22                NaN      NaN    NaN        NaN         NaN   
2018-08-23                NaN      NaN    NaN        NaN         NaN   
2018-08-24                NaN      NaN    NaN        NaN         NaN   
2018-08-25                NaN      NaN    NaN        NaN         NaN   
2018-08-26                NaN      NaN    NaN        NaN         NaN   

Country                                                      ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud  Est Hauts-Bassins Nord  ...     Hajjah   
Indicator           rCSI       rCSI rCSI          rCSI rCSI  ...       rCSI   
Datetime                                                     ...              
2018-08-22           NaN        NaN  NaN           NaN  NaN  ...  66.698570   
2018-08-23           NaN        NaN  NaN           NaN  NaN  ...  61.331133   
2018-08-24           NaN        NaN  NaN           NaN  NaN  ...  62.286205   
2018-08-25           NaN        NaN  NaN           NaN  NaN  ...  59.525377   
2018-08-26           NaN        NaN  NaN           NaN  NaN  ...  59.384136   

Country                                                                        \
AdminStrata        Ibb       Lahj      Marib     Raymah     Sa'ada     Sana'a   
Indicator         rCSI       rCSI       rCSI       rCSI       rCSI       rCSI   
Datetime                                                                        
2018-08-22   60.797456  44.782684  60.490004  56.301100  46.435518  52.667849   
2018-08-23   61.011650  44.038726  59.568557  57.324106  45.261947  50.214606   
2018-08-24   62.032249  43.296925  58.277608  57.417614  44.520462  50.073157   
2018-08-25   60.993741  42.149079  56.908462  55.324169  46.345940  48.791655   
2018-08-26   58.800240  42.146629  57.722567  57.666522  43.294599  48.839640   

Country                                       
AdminStrata    Shabwah    Socotra      Taizz  
Indicator         rCSI       rCSI       rCSI  
Datetime                                      
2018-08-22   34.302200  38.303104  50.919038  
2018-08-23   36.742499  39.545071  50.415852  
2018-08-24   35.857748  41.867782  49.391298  
2018-08-25   34.658434  48.585591  50.302392  
2018-08-26   35.913080  48.662394  50.293046  

[5 rows x 57 columns]

In [18]:
def plot(country, graph, df, save, name):
    group = df[country]
    # Adjust time-series group.
    group.columns = group.columns.droplevel(1)
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    
    if graph == "Time-series":
        # Create figure.
        # Set default trace colors with colorway.
        colorway = sns.color_palette("hls", 8).as_hex()
        layout = go.Layout(colorway = colorway)

        fig = go.Figure(layout = layout)

        for col in group.columns:
            fig.add_trace(go.Scatter(x = group.index, y = group[col], name = col, line = dict(width = 1.5)))

        # Edit the layout.
        fig.update_layout(title = dict(text = country, y = 0.9, x = 0.5), 
                          yaxis_title = dict(text = "% of people with rCSI >= 19", font = dict(size = 10)))
        # Add legend title.
        fig.update_layout(annotations = [dict(x = 1.12, y = 1.1, align = "right", valign = "top", text = "AdminStrata", 
                                              showarrow = False, xref = "paper", yref = "paper", xanchor = "center", 
                                              yanchor = "top")])
        # Add range slider.
        fig.update_layout(xaxis = dict(title = "Datetime", rangeselector = dict(), rangeslider = dict(visible = True), 
                                       type = "date"))

        fig.show()

    if graph == "Missing values":
        # Visualization of the missing values of the current time-series.
        miss = group.notnull().astype("int")
        fig1 = go.Figure(data = go.Heatmap(z = miss, zmin = 0, zmax = 1, x = miss.columns, y = miss.index, xgap = 3, 
                                           colorscale = [[0, "white"], [1, "black"]], showscale = False, 
                                           hoverinfo = "x+y"))
        # Edit the layout.
        fig1.update_layout(title = dict(text = country, y = 0.9, x = 0.5), coloraxis_showscale = False)
        fig1.update_layout(xaxis_title = "Adminstrata", yaxis_title = "Datetime")
        fig1.show()
    
    if save:
        # Reading the column index to indicate the corresonding indicator.
        group.columns = pd.MultiIndex.from_product([group.columns, ["rCSI"]])
        # Save group.
        group.to_csv("./time-series/" + name + country + ".csv", index_label = False)

In [19]:
w1 = widgets.ToggleButtons(options = df.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df), save = fixed(False), name = fixed("wfp_rcsi_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…

ATTENTION: the adminstratas 'Hadramaut' and 'Socotra' of the Yemen country have the same time-series of the rCSI indicator! I delete the time-series of the 'Socotra' adminstrata. 

In [20]:
df = df.drop("Socotra", axis = 1, level = 1)

## Adjusting the time-series

In [21]:
# I get the time-series for each country without nan values in the middle and with equal start and end between adminstrata of the same country.
def interpolation(group):   
    group.columns = group.columns.droplevel()
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    group = group.interpolate(method = "linear", limit = 7)
    # Delete time-series that still have some NaN values.
    group.dropna(inplace = True, axis = 1)
    return group

df_interpolate = df.groupby(axis = 1, level = 0).apply(interpolation)
df_interpolate.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                rCSI     rCSI   rCSI       rCSI        rCSI   
Datetime                                                               
2018-08-22                NaN      NaN    NaN        NaN         NaN   
2018-08-23                NaN      NaN    NaN        NaN         NaN   
2018-08-24                NaN      NaN    NaN        NaN         NaN   
2018-08-25                NaN      NaN    NaN        NaN         NaN   
2018-08-26                NaN      NaN    NaN        NaN         NaN   

Country                                                      ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud  Est Hauts-Bassins Nord  ...  Hadramaut   
Indicator           rCSI       rCSI rCSI          rCSI rCSI  ...       rCSI   
Datetime                                                     ...              
2018-08-22           NaN        NaN  NaN           NaN  NaN  ...  38.303104   
2018-08-23           NaN        NaN  NaN           NaN  NaN  ...  39.545071   
2018-08-24           NaN        NaN  NaN           NaN  NaN  ...  41.867782   
2018-08-25           NaN        NaN  NaN           NaN  NaN  ...  48.585591   
2018-08-26           NaN        NaN  NaN           NaN  NaN  ...  48.662394   

Country                                                                        \
AdminStrata     Hajjah        Ibb       Lahj      Marib     Raymah     Sa'ada   
Indicator         rCSI       rCSI       rCSI       rCSI       rCSI       rCSI   
Datetime                                                                        
2018-08-22   66.698570  60.797456  44.782684  60.490004  56.301100  46.435518   
2018-08-23   61.331133  61.011650  44.038726  59.568557  57.324106  45.261947   
2018-08-24   62.286205  62.032249  43.296925  58.277608  57.417614  44.520462   
2018-08-25   59.525377  60.993741  42.149079  56.908462  55.324169  46.345940   
2018-08-26   59.384136  58.800240  42.146629  57.722567  57.666522  43.294599   

Country                                       
AdminStrata     Sana'a    Shabwah      Taizz  
Indicator         rCSI       rCSI       rCSI  
Datetime                                      
2018-08-22   52.667849  34.302200  50.919038  
2018-08-23   50.214606  36.742499  50.415852  
2018-08-24   50.073157  35.857748  49.391298  
2018-08-25   48.791655  34.658434  50.302392  
2018-08-26   48.839640  35.913080  50.293046  

[5 rows x 53 columns]

In [22]:
w1 = widgets.ToggleButtons(options = df_interpolate.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df_interpolate), save = fixed(True), name = fixed("wfp_rcsi_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…

## Fit of the time-series (smooth data)

In [23]:
def fit(group):   
    group.columns = group.columns.droplevel()
    # Delete time-series that still have some NaN values.
    group.dropna(inplace = True, axis = 0)
    
    def smooth(serie):
        bspl = splrep(np.arange(0, len(serie)), serie.values, s = 500)
        bspl_y = splev(np.arange(0, len(serie)), bspl)
        return pd.Series(bspl_y, index = serie.index, name = serie.name)
    
    group_fit = group.apply(smooth)

    return group_fit

df_fit = df_interpolate.groupby(axis = 1, level = 0).apply(fit)
df_fit.dropna(axis = 0, how = "all", inplace = True)
df_fit.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                rCSI     rCSI   rCSI       rCSI        rCSI   
Datetime                                                               
2018-08-22                NaN      NaN    NaN        NaN         NaN   
2018-08-23                NaN      NaN    NaN        NaN         NaN   
2018-08-24                NaN      NaN    NaN        NaN         NaN   
2018-08-25                NaN      NaN    NaN        NaN         NaN   
2018-08-26                NaN      NaN    NaN        NaN         NaN   

Country                                                      ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud  Est Hauts-Bassins Nord  ...  Hadramaut   
Indicator           rCSI       rCSI rCSI          rCSI rCSI  ...       rCSI   
Datetime                                                     ...              
2018-08-22           NaN        NaN  NaN           NaN  NaN  ...  38.284427   
2018-08-23           NaN        NaN  NaN           NaN  NaN  ...  39.507179   
2018-08-24           NaN        NaN  NaN           NaN  NaN  ...  42.669887   
2018-08-25           NaN        NaN  NaN           NaN  NaN  ...  46.369484   
2018-08-26           NaN        NaN  NaN           NaN  NaN  ...  49.495563   

Country                                                                        \
AdminStrata     Hajjah        Ibb       Lahj      Marib     Raymah     Sa'ada   
Indicator         rCSI       rCSI       rCSI       rCSI       rCSI       rCSI   
Datetime                                                                        
2018-08-22   66.091624  61.262512  44.935179  59.832984  57.021992  46.413086   
2018-08-23   62.907892  61.026982  44.001270  59.191544  56.259488  45.289176   
2018-08-24   60.941858  60.759057  43.280679  58.645865  56.375311  44.912569   
2018-08-25   59.949845  60.467450  42.755336  58.189831  57.110285  44.938988   
2018-08-26   59.688175  60.160874  42.407171  57.817328  58.205234  45.024155   

Country                                       
AdminStrata     Sana'a    Shabwah      Taizz  
Indicator         rCSI       rCSI       rCSI  
Datetime                                      
2018-08-22   52.663754  34.626848  50.697084  
2018-08-23   50.458883  36.015687  50.401694  
2018-08-24   49.325792  35.981301  50.180555  
2018-08-25   49.030883  35.386033  50.028495  
2018-08-26   49.340559  35.092227  49.940341  

[5 rows x 53 columns]

In [24]:
w1 = widgets.ToggleButtons(options = df_fit.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df_fit), save = fixed(True), name = fixed("wfp_rcsi_smooth_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…

### Example fit operation

In [25]:
Country = "Yemen"

In [26]:
subdf_fit = df_fit[Country]
subdf_fit.columns = subdf_fit.columns.droplevel(1)

In [27]:
subdf_interpolate = df_interpolate[Country]
subdf_interpolate.columns = subdf_interpolate.columns.droplevel(1)

In [28]:
# Create figure.
fig = go.Figure()

colorway = sns.color_palette("hls", 8).as_hex()

for i, col in enumerate(subdf_fit.columns):
    fig.add_trace(go.Scatter(x = subdf_fit.index, y = subdf_fit[col], name = col, line = dict(width = 1.5, 
                                                                                              color = colorway[i % len(colorway)]), 
                             legendgroup = col))
    fig.add_trace(go.Scatter(x = subdf_interpolate.index, y = subdf_interpolate[col], name = col, line = dict(width = 1., 
                                                                                                              color = colorway[i % len(colorway)]), 
                             showlegend = False, legendgroup = col))

# Edit the layout.
fig.update_layout(title = dict(text = Country, y = 0.9, x = 0.5), 
                  yaxis_title = dict(text = "% of people with rCSI >= 19", font = dict(size = 10)))
# Add legend title.
fig.update_layout(annotations = [dict(x = 1.12, y = 1.1, align = "right", valign = "top", text = "AdminStrata", 
                                      showarrow = False, xref = "paper", yref = "paper", xanchor = "center", 
                                      yanchor = "top")])
# Add range slider.
fig.update_layout(xaxis = dict(title = "Datetime", rangeselector = dict(), rangeslider = dict(visible = True), 
                               type = "date"))

fig.show()

## Resampling datetime

In [30]:
mask = df.index.map(lambda x: x.day) == 1
df_resample = df_interpolate[mask]
# Resampling according to the survey date monthly.
df_resample.index = df_resample.index - pd.offsets.MonthEnd() - pd.offsets.MonthBegin() + pd.offsets.MonthEnd()
df_resample.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                rCSI     rCSI   rCSI       rCSI        rCSI   
Datetime                                                               
2018-08-31                NaN      NaN    NaN        NaN         NaN   
2018-09-30                NaN      NaN    NaN        NaN         NaN   
2018-10-31                NaN      NaN    NaN        NaN         NaN   
2018-11-30                NaN      NaN    NaN        NaN         NaN   
2018-12-31                NaN      NaN    NaN        NaN         NaN   

Country                                                      ...      Yemen  \
AdminStrata Centre-Ouest Centre-Sud  Est Hauts-Bassins Nord  ...  Hadramaut   
Indicator           rCSI       rCSI rCSI          rCSI rCSI  ...       rCSI   
Datetime                                                     ...              
2018-08-31           NaN        NaN  NaN           NaN  NaN  ...  57.881262   
2018-09-30           NaN        NaN  NaN           NaN  NaN  ...  59.243628   
2018-10-31           NaN        NaN  NaN           NaN  NaN  ...  62.046022   
2018-11-30           NaN        NaN  NaN           NaN  NaN  ...  37.457083   
2018-12-31           NaN        NaN  NaN           NaN  NaN  ...  43.605432   

Country                                                                        \
AdminStrata     Hajjah        Ibb       Lahj      Marib     Raymah     Sa'ada   
Indicator         rCSI       rCSI       rCSI       rCSI       rCSI       rCSI   
Datetime                                                                        
2018-08-31   59.251571  57.726031  41.652889  58.205365  59.693653  41.128674   
2018-09-30   71.581258  65.790253  46.544588  56.315925  51.152298  47.648681   
2018-10-31   78.341790  61.492933  43.906370  50.754266  71.322330  56.478183   
2018-11-30   50.513849  66.666091  30.903395  70.124427  73.431880  62.503393   
2018-12-31   57.735698  61.800427  37.207329  54.360746  56.368159  52.427530   

Country                                       
AdminStrata     Sana'a    Shabwah      Taizz  
Indicator         rCSI       rCSI       rCSI  
Datetime                                      
2018-08-31   48.418105  42.039370  51.128584  
2018-09-30   64.657082  35.825966  57.690779  
2018-10-31   60.483513  41.764129  67.335267  
2018-11-30   63.117145  35.775449  48.611067  
2018-12-31   54.360449  37.063957  58.777745  

[5 rows x 53 columns]

In [31]:
w1 = widgets.ToggleButtons(options = df_resample.columns.get_level_values(0).unique(), description = "Country:", disabled = False)
w2 = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)
p = interact(plot, country = w1, graph = w2, df = fixed(df_resample), style = fixed("lines+markers"), save = fixed(False), name = fixed("wfp_fcs_resample_"))

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…